In [1]:
# File location and type
file_location = "/FileStore/tables/Dept.xml"
file_type = "xml"

# CSV options
infer_schema = "false"
first_row_is_header = "false"
delimiter = ","


dept_df=spark.read.format("com.databricks.spark.xml").option("rowTag", "dept").load("dbfs:/FileStore/tables/Dept.xml")

Department = dept_df.select('dept_id', 'dept_name')
Department.show()

emp_df = dept_df.withColumn("employee", explode(col("employee")))
Employee = emp_df.select(emp_df.dept_id, emp_df.employee.emp_id, concat(emp_df.employee.emp_fname,lit(" ") ,emp_df.employee.emp_lname) , emp_df.employee.emp_title, emp_df.employee.emp_ssn)
Employee.show()

emp_df = emp_df.withColumn("address", explode(emp_df.employee.address))
Address = emp_df.select(emp_df.employee.emp_id, emp_df.address.street_1, emp_df.address.street_2, emp_df.address.city, emp_df.address.state,       emp_df.address.zipcode,  emp_df.address.start_date,  emp_df.address.end_date)
Address.show()

+-------+---------------+
dept_id| dept_name|
+-------+---------------+
 1| Marketing|
 2| Sales|
 3| Engineering|
 4|Human Resources|
+-------+---------------+

+-------+---------------+---------------------------------------------------+------------------+----------------+
dept_id|employee.emp_id|concat(employee[emp_fname], , employee[emp_lname])|employee.emp_title|employee.emp_ssn|
+-------+---------------+---------------------------------------------------+------------------+----------------+
 1| 1| Chandler Bing| Director| 123-45-6789|
 1| 2| Joey Tribbiani| Manager| 234-56-7890|
 1| 3| Ross Geller| Associate| 345-67-8901|
 2| 4| Monica Geller| Global VP| 456-78-9012|
 2| 5| Phoebe Buffay| Manager - US East| 567-89-0123|
 2| 6| Rachel Green| Sales Director| 678-90-1234|
 3| 7| Quentin Tarantino| Director| 789-10-2345|
 3| 8| Bradley Pitt| Manager| 890-12-3456|
 3| 9| Leonardo DiCaprio| Engineer| 901-23-4567|
 4| 10| Margot Robbie| EVP| 012-34-5678|
 4| 11| Robert Redford| Recruiting| 987-65-4321|
+-------+---------------+---------------------------------------------------+------------------+----------------+

+---------------+--------------------+----------------+------------+-------------+---------------+------------------+----------------+
employee.emp_id| address.street_1|address.street_2|address.city|address.state|address.zipcode|address.start_date|address.end_date|
+---------------+--------------------+----------------+------------+-------------+---------------+------------------+----------------+
 1|12345 Central Par...| | Hazel Crest| IL| 60429| 1968-06-01| 1976-05-01|
 1| 1011 Spencer Ave.| |Indianapolis| IN| 46218| 1976-05-02| 1980-08-31|
 1| 4300 36th Street S.| | Arlington| VA| 22206| 1980-09-01| |
 2| 2312 28th Rd. S.| | Arlington| VA| 22206| 2000-03-01| 2012-12-31|
 2| 4102 29th Street S.| | Arlington| VA| 22206| 2013-01-01| 2017-08-31|
 2| 1501 5th Avenue| #305| New York| NY| 10012| 2017-09-01| |
 3| 123 Burbank Dr| |Orchard Park| NY| 14127| 2013-05-07| 2017-08-31|
 3| 27 Brantford Place| | Buffalo| NY| 14222| 2017-09-01| |
 4| 123 Oak St.| | Brooklyn| NY| 11223| 1997-01-01| 2011-08-30|
 4| 54321 Willow Ln.| | Queens| NY| 11265| 2011-09-01| 2015-07-18|
 4| 52 Longmeadow Dr.| | Southampton| NY| 11713| 2015-07-19| |
 5| 123 Howard Street| Apt #712| New York| NY| 10013| 1997-01-01| 2011-08-30|
 5|30 Rockefeller Plaza| 9th Floor| New York| NY| 10036| 2011-09-01| 2015-07-18|
 5| 1616 Flatbush Ave.| | Queens| NY| 11265| 2015-07-19| |
 6|7200 Central Park...| | New York| NY| 10023| 1997-01-01| 2011-08-30|
 6| 450 Concar Ave| | San Mateo| CA| 94402| 2011-09-01| 2015-07-18|
 6|12345 Meadowbrook...| | Westfield| IN| 46038| 2015-07-19| 2019-06-30|
 6| 15 Campus Dr.| Apt 102| Bloomington| IN| 44444| 2019-07-01| |
 7| 77 Sunset Strip| | Hollywood| CA| 90038| 1997-01-01| 2011-08-30|
 7|10202 Washington ...| | Culver City| CA| 90232| 2011-09-01| 2015-07-18|
+---------------+--------------------+----------------+------------+-------------+---------------+------------------+----------------+
only showing top 20 rows

In [2]:
create or replace table Department
(
   dept_id int ,
    dept_name varchar(20),
    primary key(dept_id)
);

create or replace table Address
  ( emp_id int not null,
    add_history int not null autoincrement,
    street_1 varchar(100),
    street_2 varchar(50),
    city varchar(20),
    state varchar(10),
    zipcode int,
    start_date date,
    end_date date,
    foreign key(emp_id) REFERENCES Employee(emp_id)
  );
  
  
  
create or replace table Employee(
    dept_id int,
    emp_id int,
    emp_name varchar(30),
    emp_title varchar(30),
    emp_ssn varchar(30),
    primary key(emp_id),
     FOREIGN KEY (dept_id) REFERENCES Department(dept_id)
 );